In [1]:
from pyclbr import Class
from networkx.algorithms.centrality import group


# from CLS_thesis.static_network.node_initialization import hash_groups
from itertools import product
import math
import random
import pandas as pd
import networkx as nx
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
from pathlib import Path
import time

import sys

sys.path.append('../')
# C:\Users\KGulp\Documents\Computational Science\CLS_thesis\static_network\node_initialization.py
# from CLS_thesis.static_network.node_initialization import hash_groups
from static_network import node_initialization

In [2]:

def hash_groups():
    '''
    Making a hash dictionary based on the groups
    Making the hash and rehash dictionaries
    '''

    # Initializing hash dictionaries
    hash_dict = {}
    rehash_dict = {}

    # Read oplniv dataframe
    df = pd.read_csv('../Data/tab_n_(with oplniv).csv')

    # Hash every group
    for i in range(df.shape[0]):
        group = df.iloc[i]
                
        age = group['lft']
        etnc = group['etngrp']
        gender = group['geslacht']
        education = group['oplniv']


        hash_dict[f'{age}, {etnc}, {gender}, {education}'] = i
        rehash_dict[i] = f'{age}, {etnc}, {gender}, {education}' 
    
    return hash_dict, rehash_dict

def spatial(group_dict, group_nodes):
    '''
    Makes 2 dictionaries: areas_groups, node_area

    areas_group: is the dictionary which takes as key area and group and returns the nodes in that area of the group
    node_area: returns a dictionary whith as key the node and value the area in which the node is present

    '''
    a = 0
    b = 0
    
    print(len(group_nodes))
    # Import spatial data (see spatial data.ipynb to see how csv is constructed)
    df_sp = pd.read_csv('..\Data\Spatial_data\spatial_data_22gebieden.csv')

    # get unique areas and etnicities and groups
    group_list = df_sp.groupby(['geslacht','etngrp', 'oplniv']).size().reset_index().to_numpy()[:, [0, 1, 2]]
    area_list = df_sp['code'].unique()
    
    
    print(len(group_list))
    groups = list(group_nodes.keys())

    # Initialize Dictionaries
    areas_groups = {}
    node_area = {}
    areas_df = {} 
    areas = {}

    # Loop through area's and initialize areas with a second dictionary
    for area in area_list:
        areas[area] = {}
        areas_df[area] = {}
        areas_groups[area] = {}

        # Give areas df a value (max amount of people represented in that area)
        # Initialize areas and group_areas with a list for each area
        for group in group_list:
            gender, etn, edu = group
            
            amount = df_sp[(df_sp['geslacht'] == gender) & (df_sp['oplniv'] == edu) & (df_sp['etngrp'] == etn)]
            areas_df[area][tuple(group)] = float(amount[amount['code'] == area]['percentage']/100)
            areas[area][tuple(group)] = []
            
        for group in groups:
            areas_groups[area][group] = []
    
    # Loop through the groups 
    for group in group_dict:
        
        group = tuple(group)
        # Shuffle areas and nodes to put the nodes randomly in the areas
        random.shuffle(group_dict[group])
        random.shuffle(area_list)

        # Initialize area and iterator
        area = 0
        i = 0
 
        x = True
        

        # Loop through all nodes with ethnicity etn
        while i < len(group_dict[group]):
            
            
            if area == 1 and x == True:
                x = False
                a += 1
                
                print(a)
                
#             print(len(group_dict[group]), i, group)
            # node is i_th node of the list
            
            node = group_dict[group][i]
            
            # Go to next area
            
#             print(len(group_dict[group])* areas_df[area][group])
            
    
            # Check if area is full and area is not greater than 22
            if len(areas[area][group]) < math.ceil(areas_df[area][group] * sum(areas_df[area].values())) and area < 22:

                # Append node to area
                areas[area][group].append(node)
                areas_groups[area][node[1]].append(node[0])
                node_area[node[0]] = area
                i+=1
                
            
      
            else:
                area+=1
                
                
                if area > 21:
                    
                    b += 1
                    print(b)
                    
                    rest = len(group_dict[group]) - i
                    
                    print(i/len(group_dict[group]))
                    tot = i
                    
                    for j in area_list:
                        percentage = areas_df[j][group]/tot
                        
                        areas_df[j][group] += math.ceil(rest * percentage)
                    
                    area = 0
                

    return areas_groups, node_area
def initialize_nodes(df, hash_dict):
    '''
    Initilizes all the nodes for the network
    '''

    # Initialize list with nodes and nodes per group (example: Man,"[0,20)",1,Autochtoon)
    all_nodes = []
    group_nodes = {}
    df_sp = pd.read_csv('..\Data\Spatial_data\spatial_data_22gebieden.csv')
    
    group_list = df_sp.groupby(['geslacht','etngrp', 'oplniv']).size().reset_index().to_numpy()[:, [0, 1, 2]]
    print(group_list)
    group_dict = {tuple(group) : [] for group in group_list}
    

    id = 0

    # Loops through all lines in the tab_n file and get group properties of each line
    for i in range(df.shape[0]):
        group = df.iloc[i]

        
        age = group['lft']
        etnc = group['etngrp']
        gender = group['geslacht']
        education = group['oplniv']
        nodes = []
        
        hd = hash_dict[f'{age}, {etnc}, {gender}, {education}']
        
         
        # Makes n (the size of the group) nodes of the group
        for _ in range(int(group['n'])):
             
            node = (id, hd)
            nodes.append(node[0])
            all_nodes.append(id)
            group_dict[tuple([gender, etnc, education])].append(node)

            id += 1
            
   
                
                
        # make a dictionary with as key the group properties and value a list of nodes of the group
        group_nodes[hd] = nodes




    return all_nodes, group_nodes, group_dict


In [3]:
'''
Hash function
'''

hash_dict, rehash_dict = hash_groups()

'''
Initialize nodes 
'''

df_nodes = pd.read_csv('../Data/tab_n(with oplniv).csv')
all_nodes, group_nodes, etn_dict = initialize_nodes(df_nodes, hash_dict)


area_dict, node_area = spatial(etn_dict, group_nodes)




[['Man' 'Autochtoon' 1]
 ['Man' 'Autochtoon' 2]
 ['Man' 'Autochtoon' 3]
 ['Man' 'Marokkaans' 1]
 ['Man' 'Marokkaans' 2]
 ['Man' 'Marokkaans' 3]
 ['Man' 'Overig' 1]
 ['Man' 'Overig' 2]
 ['Man' 'Overig' 3]
 ['Man' 'Surinaams' 1]
 ['Man' 'Surinaams' 2]
 ['Man' 'Surinaams' 3]
 ['Man' 'Turks' 1]
 ['Man' 'Turks' 2]
 ['Man' 'Turks' 3]
 ['Vrouw' 'Autochtoon' 1]
 ['Vrouw' 'Autochtoon' 2]
 ['Vrouw' 'Autochtoon' 3]
 ['Vrouw' 'Marokkaans' 1]
 ['Vrouw' 'Marokkaans' 2]
 ['Vrouw' 'Marokkaans' 3]
 ['Vrouw' 'Overig' 1]
 ['Vrouw' 'Overig' 2]
 ['Vrouw' 'Overig' 3]
 ['Vrouw' 'Surinaams' 1]
 ['Vrouw' 'Surinaams' 2]
 ['Vrouw' 'Surinaams' 3]
 ['Vrouw' 'Turks' 1]
 ['Vrouw' 'Turks' 2]
 ['Vrouw' 'Turks' 3]]
240
30
1
1
0.0004067596050734016
2
2
0.00468204965584137
3
3
0.004429387325313891
4
4
0.005817358372701363
5
5
0.006313131313131313
6
6
0.02612671456564337
7
7
0.011554326427299148
8
8
0.007693812349220838
9
9
0.005058031883665267
10
10
0.010237686139747995
11
11
0.006151682924685825
12
12
0.0151378219611387

In [4]:
df_sp = pd.read_csv('..\Data\Spatial_data\spatial_data2.csv')

df_sp.groupby(by='code').sum()

FileNotFoundError: [Errno 2] No such file or directory: '..\\Data\\Spatial_data\\spatial_data2.csv'

In [5]:
df = pd.read_csv('../Data/Spatial_data/bbga_latest_and_greatest_948e83a9fa.csv', sep=';')

In [6]:
df.columns

Index(['jaar', 'gebiedcode15', 'variabele', 'waarde'], dtype='object')

In [7]:
df = df[df['jaar'] == 2019]



In [8]:
possibilities = list(df['variabele'].unique())

for possible in possibilities:

    if possible[-2:] == '_P' and possible[0:3] == 'BEV':
        print(possible)

BEVMAN_P
BEVVROUW_P
BEVINST_P
BEVINST65PLUS_P
BEVPOTBBV15_74_P
BEVSUR_P
BEVANTIL_P
BEVTURK_P
BEVMAROK_P
BEVOVNW_P
BEVWEST_P
BEVAUTOCH_P
BEVNW_P
BEVNSTEDELING_P
BEVEENOUDERHH_P
BEVALLEENHH_P
BEVPAARZKINDHH_P
BEVPAARMKINDHH_P
BEVOVERIGHH_P
BEVHHMKIND_P
BEV1P75PLUS_P
BEVMUTNL_P
BEVMUTTOT_P
BEV0_3_P
BEV65PLUS_P
BEV4_12_P
BEV13_17_P
BEV0_17_P
BEV0_18_P
BEV18_26_P
BEV18_64_P
BEV27_65_P
BEV66PLUS_P
BEV65_74_P
BEV75PLUS_P
BEV80PLUS_P
BEV17AUTO_P
BEV18_22NL_P
BEV23_39NL_P
BEV40_64NL_P
BEV65PLUSNL_P
BEV65_79NL_P
BEV80PLUSNL_P
BEV17W_P
BEV18_22W_P
BEV23_39W_P
BEV40_64W_P
BEV65PLUSW_P
BEV65_79W_P
BEV80PLUSW_P
BEV17NW_P
BEV18_22NW_P
BEV23_39NW_P
BEV40_64NW_P
BEV65PLUSNW_P
BEV65_79NW_P
BEV80PLUSNW_P


In [25]:
groepen = ['BEVSUR_P',
'BEVANTIL_P',
'BEVTURK_P',
'BEVMAROK_P',
'BEVOVNW_P',
'BEVWEST_P',
'BEVAUTOCH_P',
]

groepen = ['BEVMAN_P', 'BEVVROUW_P']

# groepen = ['BEVOPLLAAG_P',
# 'BEVOPLMID_P',
# 'BEVOPLHOOG_P']


# groepen  = ['BEV0_17']

In [26]:
x = 0

wijken = []
for i in df['gebiedcode15'].unique():
#     print(i)
    
    if i == 'A00':
        x = 1
    
    if i == 'Z99':
        break
        
    if x == 1:
        wijken.append(i)

print(wijken)

['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'B10', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20', 'E21', 'E22', 'E36', 'E37', 'E38', 'E39', 'E40', 'E41', 'E42', 'E43', 'E75', 'F11', 'F76', 'F77', 'F78', 'F79', 'F80', 'F81', 'F82', 'F83', 'F84', 'F85', 'F86', 'F87', 'F88', 'F89', 'K23', 'K24', 'K25', 'K26', 'K44', 'K45', 'K46', 'K47', 'K48', 'K49', 'K52', 'K53', 'K54', 'K59', 'K90', 'K91', 'M27', 'M28', 'M29', 'M30', 'M31', 'M32', 'M33', 'M34', 'M35', 'M50', 'M51', 'M55', 'M56', 'M57', 'M58', 'N60', 'N61', 'N62', 'N63', 'N64', 'N65', 'N66', 'N67', 'N68', 'N69', 'N70', 'N71', 'N72', 'N73', 'N74', 'T92', 'T93', 'T94', 'T95', 'T96', 'T97', 'T98']


In [11]:
df_sp = df[df['jaar'] == 2019]


In [12]:
df_sp = df_sp.loc[df['variabele'].isin(groepen)]

In [13]:
df_sp = df_sp.loc[df['gebiedcode15'].isin(wijken)]

In [14]:
df_sp

,jaar,gebiedcode15,variabele,waarde
341214,2019,A00,BEVSUR_P,2.0
341215,2019,A01,BEVSUR_P,1.7
341216,2019,A02,BEVSUR_P,0.8
341217,2019,A03,BEVSUR_P,0.8
341218,2019,A04,BEVSUR_P,2.7
...,...,...,...,...
438634,2019,T94,BEVAUTOCH_P,21.4
438635,2019,T95,BEVAUTOCH_P,47.0
438636,2019,T96,BEVAUTOCH_P,27.8
438637,2019,T97,BEVAUTOCH_P,35.8


In [15]:
sum(df_sp[df_sp['gebiedcode15'] == 'M50']['waarde'])



nan

In [16]:
df_t = df[df['variabele'] == 'BEVDICHT']

df_t = df_t[df_t['jaar'] == 2019]
df_t = df_t.loc[df['gebiedcode15'].isin(wijken)]

df_t['amount_of_persons'] = (df_t['waarde']/df_t.waarde.sum()) * 861000

In [17]:
df_t

,jaar,gebiedcode15,variabele,waarde,amount_of_persons
102107,2019,A00,BEVDICHT,12321.0,9543.378170
102108,2019,A01,BEVDICHT,7367.0,5706.198115
102109,2019,A02,BEVDICHT,14234.0,11025.115240
102110,2019,A03,BEVDICHT,10355.0,8020.589315
102111,2019,A04,BEVDICHT,13951.0,10805.914199
...,...,...,...,...,...
102201,2019,T94,BEVDICHT,7954.0,6160.865998
102202,2019,T95,BEVDICHT,1374.0,1064.248162
102203,2019,T96,BEVDICHT,7068.0,5474.604083
102204,2019,T97,BEVDICHT,6289.0,4871.220299


In [18]:
df_sp = df_sp.replace(['BEVANTIL_P',
'BEVOVNW_P',
'BEVWEST_P'], 'OVERIG')


In [19]:
df_sp = df_sp.groupby(['variabele', 'gebiedcode15'], as_index=False).sum()

In [20]:
df_sp = df_sp.replace(['BEVAUTOCH_P','BEVSUR_P','BEVMAROK_P','BEVTURK_P', 'OVERIG' ],['Autochtoon','Surinaams','Marokkaans','Turks', 'Overig'])

In [630]:
df_sp = df_sp.replace(['BEVMAN_P','BEVVROUW_P'],['Man','Vrouw'])

In [893]:
df_sp = df_sp.replace(['BEVOPLLAAG_P','BEVOPLMID_P','BEVOPLHOOG_P'],[1,2,3])

In [924]:
df_sp = df_sp.replace(['BEV0_17'],['[0,20)'])

In [904]:
df_sp = df_sp.fillna(0)

In [21]:
df_t = df_t.fillna(0)

list(df_t['amount_of_persons'].isna()).count(True)

0

In [22]:
amount = []

for i in df_sp.iterrows():
    row = i[1]
    area = row['gebiedcode15']
    waarde = row['waarde']
    etn = row['variabele']
    
    total = df_t[df_t['gebiedcode15'] == area]['amount_of_persons']
    
    print(total)

    amount.append(math.ceil((float(total)*float(waarde)/100)) )

df_sp['amount_per_area'] = amount

print(sum(amount))

102107    9543.37817
Name: amount_of_persons, dtype: float64
102108    5706.198115
Name: amount_of_persons, dtype: float64
102109    11025.11524
Name: amount_of_persons, dtype: float64
102110    8020.589315
Name: amount_of_persons, dtype: float64
102111    10805.914199
Name: amount_of_persons, dtype: float64
102112    13499.066208
Name: amount_of_persons, dtype: float64
102113    18073.629268
Name: amount_of_persons, dtype: float64
102114    10155.282135
Name: amount_of_persons, dtype: float64
102115    7823.850572
Name: amount_of_persons, dtype: float64
102116    10441.095506
Name: amount_of_persons, dtype: float64
102117    6.971058
Name: amount_of_persons, dtype: float64
102118    4721.729837
Name: amount_of_persons, dtype: float64
102119    6307.258213
Name: amount_of_persons, dtype: float64
102120    21562.256431
Name: amount_of_persons, dtype: float64
102121    3714.799262
Name: amount_of_persons, dtype: float64
102122    18017.860806
Name: amount_of_persons, dtype: float64
10212

102140    6184.102858
Name: amount_of_persons, dtype: float64
102141    9574.360649
Name: amount_of_persons, dtype: float64
102142    881.451535
Name: amount_of_persons, dtype: float64
102143    168.854512
Name: amount_of_persons, dtype: float64
102144    8027.560373
Name: amount_of_persons, dtype: float64
102145    11791.157039
Name: amount_of_persons, dtype: float64
102146    8280.84214
Name: amount_of_persons, dtype: float64
102147    7603.100407
Name: amount_of_persons, dtype: float64
102148    5545.089223
Name: amount_of_persons, dtype: float64
102149    7438.893267
Name: amount_of_persons, dtype: float64
102150    5864.983321
Name: amount_of_persons, dtype: float64
102151    2360.090357
Name: amount_of_persons, dtype: float64
102152    6630.250559
Name: amount_of_persons, dtype: float64
102153    1444.558095
Name: amount_of_persons, dtype: float64
102154    18213.824987
Name: amount_of_persons, dtype: float64
102155    18684.758671
Name: amount_of_persons, dtype: float64
102156  

102173    20900.005937
Name: amount_of_persons, dtype: float64
102174    8622.423974
Name: amount_of_persons, dtype: float64
102175    9668.082649
Name: amount_of_persons, dtype: float64
102176    1181.207021
Name: amount_of_persons, dtype: float64
102177    10719.163257
Name: amount_of_persons, dtype: float64
102178    0.0
Name: amount_of_persons, dtype: float64
102179    9774.972202
Name: amount_of_persons, dtype: float64
102180    5634.938413
Name: amount_of_persons, dtype: float64
102181    4473.870003
Name: amount_of_persons, dtype: float64
102182    2376.356158
Name: amount_of_persons, dtype: float64
102183    1334.570294
Name: amount_of_persons, dtype: float64
102184    6354.506493
Name: amount_of_persons, dtype: float64
102185    5464.534777
Name: amount_of_persons, dtype: float64
102186    6394.783716
Name: amount_of_persons, dtype: float64
102187    8651.857329
Name: amount_of_persons, dtype: float64
102188    3729.515939
Name: amount_of_persons, dtype: float64
102189    5772

In [24]:
df_sp['area_code'] = [i%99 for i in range(495)]

In [927]:
df_sp

,jaar,gebiedcode15,variabele,waarde,amount_per_area,area_code
1407740,2021,A00,"[0,20)",245.0,24457,0
1407741,2021,A01,"[0,20)",199.0,10987,1
1407742,2021,A02,"[0,20)",566.0,61855,2
1407743,2021,A03,"[0,20)",543.0,43214,3
1407744,2021,A04,"[0,20)",866.0,92238,4
...,...,...,...,...,...,...
1407834,2021,T94,"[0,20)",6054.0,375270,94
1407835,2021,T95,"[0,20)",436.0,4813,95
1407836,2021,T96,"[0,20)",3726.0,200891,96
1407837,2021,T97,"[0,20)",2093.0,99380,97


In [28]:
df_n = pd.read_csv('./Data/tab_n(with oplniv).csv')

FileNotFoundError: [Errno 2] No such file or directory: './Data/tab_n(with oplniv).csv'

In [30]:
df_sp.to_csv('../Data/Spatial_data/Ethnic_data.csv')

In [931]:
edu_data = pd.read_csv('education_data.csv')

df_sp

edu_data
d = edu_data[edu_data['variabele'] == 1].copy()




y = edu_data[edu_data['variabele'] == 1]['amount_per_area']+ np.array(df_sp['waarde'])



d['amount_per_area'] += y



edu_data[edu_data['variabele'] == 3]
x = edu_data[(edu_data['variabele'] == 2) | (edu_data['variabele'] == 3)]

x
edu_data = pd.concat([x, d])

edu_data
edu_data
edu_data = edu_data.drop('waarde', axis=1 )

edu_data = edu_data.sort_values(by='area_code').reset_index()

edu_data

percentages = []
for i in range(99):
    partial = edu_data[edu_data['area_code'] == i].copy()
    
#     print((partial['amount_per_area']))
#     print(partial.amount_per_area.sum())
    percentage = list((partial['amount_per_area'] / partial.amount_per_area.sum()) * 100)
    
    percentages.extend(percentage)

    
edu_data['waarde'] = percentages

In [932]:
edu_data

,index,Unnamed: 0,jaar,gebiedcode15,variabele,area_code,amount_per_area,waarde
0,99,551974,2018,A00,2,0,2989.0,27.540772
1,0,548709,2018,A00,1,0,2175.0,20.040542
2,198,555239,2018,A00,3,0,5689.0,52.418686
3,100,551975,2018,A01,2,1,1643.0,26.402057
4,1,548710,2018,A01,1,1,1185.0,19.042263
...,...,...,...,...,...,...,...,...
292,295,555336,2018,T97,3,97,1173.0,13.576389
293,196,552071,2018,T97,2,97,2052.0,23.750000
294,197,552072,2018,T98,2,98,294.0,23.389021
295,296,555337,2018,T98,3,98,224.0,17.820207


In [495]:
edu_data.groupby(by='area_code').sum()

,index,Unnamed: 0,variabele,jaar,amount_per_area,waarde
area_code,,,,,,
0,66,66,6,6054,83043.0,100.0
1,69,69,6,6054,65423.0,100.0
2,72,72,6,6054,66181.0,100.0
3,75,75,6,6054,65285.0,100.0
4,78,78,6,6054,133141.0,100.0
5,81,81,6,6054,30652.0,100.0
6,84,84,6,6054,37043.0,100.0
7,87,87,6,6054,31203.0,100.0
8,90,90,6,6054,52111.0,100.0


In [820]:
edu_data2 = pd.read_csv('../Data/Spatial_data/education_data.csv')

edu_data2

,Unnamed: 0,variabele,gebiedcode15,jaar,waarde,amount_per_area,area_code
0,0,3,DX01,2018,62.0,43456,0
1,1,3,DX02,2018,63.0,34206,1
2,2,3,DX03,2018,53.0,27844,2
3,3,3,DX04,2018,47.0,22075,3
4,4,3,DX05,2018,55.0,57073,4
...,...,...,...,...,...,...,...
61,61,2,DX18,2018,36.0,7134,17
62,62,2,DX19,2018,37.0,1718,18
63,63,2,DX20,2018,41.0,5776,19
64,64,2,DX21,2018,42.0,14378,20


In [828]:
gender_data = pd.read_csv('gender_data.csv')

gender_data

,Unnamed: 0,jaar,gebiedcode15,variabele,waarde,amount_per_area,area_code
0,67981,2021,A00,Man,54.9,5481,0
1,67982,2021,A01,Man,55.3,3053,1
2,67983,2021,A02,Man,52.3,5716,2
3,67984,2021,A03,Man,54.3,4322,3
4,67985,2021,A04,Man,52.0,5539,4
...,...,...,...,...,...,...,...
193,84296,2021,T94,Vrouw,49.6,3075,94
194,84297,2021,T95,Vrouw,47.6,526,95
195,84298,2021,T96,Vrouw,52.0,2804,96
196,84299,2021,T97,Vrouw,50.3,2389,97


In [948]:
gender_data = pd.read_csv('gender_data.csv')

etn_data = pd.read_csv('ethnicity_data.csv')

# edu_data = pd.read_csv('../Data/Spatial_data/education_data.csv')
codes = []
etnicities = []
amounts = []
percentages = []
genders = []
educations = []

df = pd.DataFrame()
for i in etn_data.itertuples():
#     print(i)
    e = i[2]
    c = i[-1]

    amount = i[-2]
    
    waarde = i[-3]
    
    for j in ['Man', 'Vrouw']:
        
        gender = gender_data[gender_data['area_code'] == c ]
        gender = gender[gender['variabele'] == j]
        gender_waarde = float((gender['waarde'])/100)

        for k in [1,2,3]:
            
            education = edu_data[edu_data['area_code'] == c]
            education = education[education['variabele'] == k]

            education_waarde = float((education['waarde'])/100)
            
#             print(k)
#             print(education_waarde)
#             print(gender_waarde)
            waarde2 = waarde  * gender_waarde * education_waarde
  
            amount2 = amount + float(education['amount_per_area']) + float(gender['amount_per_area'])
        
            amounts.append(amount2)
            codes.append(c)
            percentages.append(waarde2)
            etnicities.append(e)
            genders.append(j)
            educations.append(k)
        
df['etngrp'] = etnicities
df['code'] = codes
df['geslacht'] = genders
df['amount'] = amounts
df['oplniv'] = educations
df['percentage'] = percentages

In [949]:
df

,etngrp,code,geslacht,amount,oplniv,percentage
0,Autochtoon,0,Man,12478.0,1,5.314090
1,Autochtoon,0,Man,13292.0,2,7.302904
2,Autochtoon,0,Man,15992.0,3,13.899706
3,Autochtoon,0,Vrouw,11499.0,1,4.365491
4,Autochtoon,0,Vrouw,12313.0,2,5.999289
...,...,...,...,...,...,...
2965,Overig,98,Man,748.0,2,1.631267
2966,Overig,98,Man,678.0,3,1.242870
2967,Overig,98,Vrouw,1220.0,1,4.424300
2968,Overig,98,Vrouw,775.0,2,1.760141


In [552]:
df.groupby(by=['etngrp', 'geslacht', 'oplniv']).sum().reset_index()


,etngrp,geslacht,oplniv,code,amount,percentage
0,Autochtoon,Man,1,231,1333170.0,210.347523
1,Autochtoon,Man,2,231,1077390.0,99.219027
2,Autochtoon,Man,3,231,1254247.0,156.521350
3,Autochtoon,Vrouw,1,231,1338065.0,214.358587
4,Autochtoon,Vrouw,2,231,1082285.0,100.882421
5,Autochtoon,Vrouw,3,231,1259142.0,158.821692
6,Marokkaans,Man,1,231,1012294.0,51.385105
7,Marokkaans,Man,2,231,756514.0,19.819386
8,Marokkaans,Man,3,231,933371.0,25.348409
9,Marokkaans,Vrouw,1,231,1017189.0,51.817258


In [184]:
df[df['oplniv'] == 1].groupby(['etngrp', 'geslacht']).mean()

code       amount  oplniv  percentage
etngrp     geslacht                                       
Autochtoon Man       10.5  1766.863636     1.0   21.185814
           Vrouw     10.5  1792.681818     1.0   21.548305
Marokkaans Man       10.5   422.045455     1.0    4.388768
           Vrouw     10.5   426.954545     1.0    4.433650
Overig     Man       10.5  1340.954545     1.0   17.470964
           Vrouw     10.5  1354.954545     1.0   17.704691
Surinaams  Man       10.5   382.318182     1.0    4.080632
           Vrouw     10.5   384.818182     1.0    4.119173
Turks      Man       10.5   247.318182     1.0    2.505841
           Vrouw     10.5   250.227273     1.0    2.530345

In [950]:
df.to_csv('spatial_data.csv')

df1 = pd.read_csv('../Data/tab_n_(with oplniv).csv')

df1[(df1['etngrp'] == 'Autochtoon') & (df1['oplniv'] == 1) & (df1['geslacht'] == 'Man')]

df1[(df1['lft'] == '[0,20)') & (df1['oplniv'] == 1)]



,Unnamed: 0,geslacht,lft,etngrp,oplniv,n
6,6,Man,"[0,20)",Autochtoon,1,30825
12,12,Vrouw,"[0,20)",Autochtoon,1,29308
39,39,Man,"[0,20)",Marokkaans,1,12205
48,48,Man,"[0,20)",Surinaams,1,5341
51,51,Vrouw,"[0,20)",Overig,1,24600
57,57,Man,"[0,20)",Overig,1,26101
75,75,Vrouw,"[0,20)",Marokkaans,1,11509
171,171,Man,"[0,20)",Turks,1,5620
195,195,Vrouw,"[0,20)",Turks,1,5047
198,198,Vrouw,"[0,20)",Surinaams,1,5056


In [110]:
df[(df['etngrp'] == 'Autochtoon') & (df['oplniv'] == 1) & (df['geslacht'] == 'Man')].amount.sum()

38871

In [24]:
# importing product
from itertools import product
  
# apply product method
x = list(product(df['geslacht'], df['etngrp'] ))

In [32]:
df.groupby(['geslacht','etngrp', 'oplniv']).size().reset_index().to_numpy()



array([['Man', 'Autochtoon', 1, 22],
       ['Man', 'Autochtoon', 2, 22],
       ['Man', 'Autochtoon', 3, 22],
       ['Man', 'Marokkaans', 1, 22],
       ['Man', 'Marokkaans', 2, 22],
       ['Man', 'Marokkaans', 3, 22],
       ['Man', 'Overig', 1, 22],
       ['Man', 'Overig', 2, 22],
       ['Man', 'Overig', 3, 22],
       ['Man', 'Surinaams', 1, 22],
       ['Man', 'Surinaams', 2, 22],
       ['Man', 'Surinaams', 3, 22],
       ['Man', 'Turks', 1, 22],
       ['Man', 'Turks', 2, 22],
       ['Man', 'Turks', 3, 22],
       ['Vrouw', 'Autochtoon', 1, 22],
       ['Vrouw', 'Autochtoon', 2, 22],
       ['Vrouw', 'Autochtoon', 3, 22],
       ['Vrouw', 'Marokkaans', 1, 22],
       ['Vrouw', 'Marokkaans', 2, 22],
       ['Vrouw', 'Marokkaans', 3, 22],
       ['Vrouw', 'Overig', 1, 22],
       ['Vrouw', 'Overig', 2, 22],
       ['Vrouw', 'Overig', 3, 22],
       ['Vrouw', 'Surinaams', 1, 22],
       ['Vrouw', 'Surinaams', 2, 22],
       ['Vrouw', 'Surinaams', 3, 22],
       ['Vrouw', 'Turk